In [1]:
# valuation_date時におけるcodeで指定した銘柄の個別銘柄分析
# SBI証券のモバイルの銘柄分析を参考に作成

In [2]:
### パラメータ設定セル
from datetime import date
from dateutil.relativedelta import relativedelta

# パラメータをセット
# code: 証券コード
# valuation_date: 過去にさかのぼって評価日をdatetime.dateでセット可能。デフォルト値は今日(date.today())
code = 2708
valuation_date = date.today()
# valuation_date = date(2024, 2, 2)


In [3]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, FinancequotePl, KessanPl, KessanFig, PricelistFig, read_data, get_df_latest_yearly_performance, get_latest_stockprice, print_finance_quote
from lib_dataprocess import get_fig_expected_performance_progress_rate_pycharts, MeigaralistPl, get_fig_actual_performance_progress_rate_pycharts, ShikihoPl
import polars as pl

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)
FPL.with_columns_market_cap()

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [4]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

久世(2708)の銘柄情報

終値: 1669.0円(2024年12月23日)
予想配当利回り: 0.9%(2024年12月23日)
予想PER: 5.94倍(2024年12月23日)
実績PBR: 1.07倍(2024年12月23日)
自己資本比率: 26.8%(2024年12月23日)
予想ROE: 17.94%(2024年12月23日)
予想ROA: 4.81%(2024年12月23日)
時価総額: 77.2億円(2024年12月23日)

2708(久世)の四季報データ履歴

発行日: 2024年09月13日
【上振れ】
  食材卸、製造は法人や訪日客需要回復で外食向け絶好調。期央の新倉庫開設で機会損失解消し、数量尻上がり。子会社売却の目減り補う。賃料、物流費、人件費増こなし一転営業増益。税負担増。減配。
【PB】
  外食店舗の人手不足に対応し、簡易な調理で提供可能な高付加価値PB商品の開発加速。26年3月期営業益19億円目指す中計は目標過小。早期の上方修正も。

発行日: 2024年06月14日
【減配】
  食材卸、製造は良好な受注環境追い風に取引先当たりの数量、取引先数ともに増え、水産輸出も堅調。が、物流機能拡充による賃料、運賃、人件費の増加こなせず営業益急落。補助金特益消える。減配。
【キャパ拡大】
  機会損失解消に向け、コロナ下で8カ所から5カ所に縮小した物流拠点を順次再拡張。新規先は飲食店に加え、遊園地やケータリングなど全方位で開拓。

発行日: 2024年03月16日
【小幅増益】
  食材卸、製造ともに飲食店の客数回復し快走。水産輸出も堅調。営業益独自再増額。特配。25年3月期も食材卸、製造ともに良好な受注環境続くが、物流キャパ限界。営業益小幅増止まり。税平常化。
【課題】
  機会損失回避に向け物流インフラ整備課題。常温、冷蔵、冷凍の3温度帯の管理ノウハウ生かした食品のネット通販事業者向け物流受託事業は慎重に育成。

発行日: 2023年12月15日
【独自増額】
  食材卸、製造ともに飲食店の客数回復し絶好調。機内食も寄与。物流投資再開による修繕費増こなす。営業益独自増額。会社増額計画なお慎重。25年3月期も食材卸、製造ともに数量堅調。営業増益。
【ＤＸ】
  LINEの拡張機能使った無料受発注システム『KUZEX』は、全取引先での導

In [5]:
### 業績推移

# KessanFig(
#    code: int, 
#    settlement_type: Literal["通期", "四半期"], 
#    output_target: str = "jupyter",
#    start_settlement_date: date = date(1900, 1, 1),
#    end_settlement_date: date = date(2999, 12, 31)
#)

KFIG = KessanFig(code, "通期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [6]:
# 今季決算予想発表の推移
df = KPL.get_settlement_forcast(code, valuation_date)
df

2024-03-31


code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
2708,2025-03-31,"""予""",2024-05-15,67000,1400,1450,1300,281.0,15.0,4


In [7]:
### 通期業績、業績予想
# def get_df_latest_yearly_performance(code: int, valuation_date: date=date.today()) -> pl.DataFrame:
df = get_df_latest_yearly_performance(code, valuation_date)
df

久世(2708)の通期決算(評価日：2024年12月24日)


決算期,決算発表日,売上高,営業利益,経常利益,純利益,EPS,1株配当
str,date,i64,i64,i64,i64,f64,f64
"""2025年3月期(予)""",2024-05-15,67000,1400,1450,1300,281.0,15.0
"""2024年3月期""",2024-05-15,64474,1861,1939,2008,434.1,42.0


In [8]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()
KPL.get_latest_quater_settlement(code, valuation_date)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
2708,2024-09-30,"""四""",2024-11-14,16416,312,423,272,58.8,1.9,2
2708,2024-06-30,"""四""",2024-08-14,17018,424,448,624,135.0,2.5,1
2708,2024-03-31,"""四""",2024-05-15,15410,54,76,64,13.8,0.4,4
2708,2023-12-31,"""四""",2024-02-14,17518,900,930,705,152.5,5.1,3
2708,2023-09-30,"""四""",2023-11-14,16063,477,484,625,135.1,3.0,2
2708,2023-06-30,"""四""",2023-08-14,15483,430,449,614,132.8,2.8,1
2708,2023-03-31,"""四""",2023-05-15,14294,257,266,282,61.4,1.8,4
2708,2022-12-31,"""四""",2023-02-10,15974,526,549,446,97.2,3.3,3


In [9]:
# codeで指定した銘柄のevaluation_dateで指定した時点での最新の年度決算予想に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# from plotly.graph_objects import Figure
# def get_fig_expected_performance_progress_rate_pycharts(code: int, evaluation_date: date=date.today()) -> Figure:

fig1 = get_fig_expected_performance_progress_rate_pycharts(code, valuation_date)
fig1.show()

久世(2708)の2025年3月期第2四半期決算進捗率(評価日：2024-12-24)


In [10]:
last_year_valuation_date = date(valuation_date.year-1, valuation_date.month, valuation_date.day)

# codeで指定した銘柄のevaluation_dateで指定した時点での最新の四半期決算発表に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# valuation_dateで指定する日において、当年度の本決算が発表されていない日を指定した場合は前期末第4四半期の進捗率を表示するので、進捗率100%で表示される。
fig2 = get_fig_actual_performance_progress_rate_pycharts(code, last_year_valuation_date, KPL.df, MPL.df)
fig2.show()

久世(2708)の2024年3月期第2四半期決算進捗率(評価日：2023-12-24)


In [11]:
#### valuation_dateに過去日を指定した場合、valuation_dateを含む期の株価推移
#### 答え合わせ

In [12]:
# 株価推移
# 前期算発表翌営業日～今期決算発表日翌営業日
start_date, end_date = KPL.get_current_settlement_period_by_announcement_date(code, valuation_date, "四")

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>start_date)
start_date = df["date"].to_list()[0]

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>end_date)
if not end_date == date(2999, 12, 31):
    end_date = df["date"].to_list()[0]

PFIG = PricelistFig(code, RevPPL.df, MPL.df, start_date, end_date)
df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")<=valuation_date)
vline_date = df["date"].to_list()[-1]
PFIG.add_vline(vline_date)
PFIG.fig.show()

In [13]:
# appendix
KPL.df.filter(pl.col("code")==code).filter(pl.col("announcement_date")>=date(2024,1,1))

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
2708,2023-12-31,"""四""",2024-02-14,17518,900,930,705,152.5,5.1,3
2708,2024-03-31,"""予""",2024-02-27,65000,1700,1730,1740,376.1,42.0,4
2708,2024-03-31,"""四""",2024-05-15,15410,54,76,64,13.8,0.4,4
2708,2025-03-31,"""予""",2024-05-15,67000,1400,1450,1300,281.0,15.0,4
2708,2024-03-31,"""本""",2024-05-15,64474,1861,1939,2008,434.1,42.0,4
2708,2024-06-30,"""四""",2024-08-14,17018,424,448,624,135.0,2.5,1
2708,2024-09-30,"""四""",2024-11-14,16416,312,423,272,58.8,1.9,2


In [14]:
PFIG = PricelistFig(code, RevPPL.df, MPL.df, date(2023, 12, 1), date.today())
PFIG.fig.show()

In [15]:
from lib_dataprocess import CreditbalancePl

CPL = CreditbalancePl()
CPL.with_columns_margin_ratio()
CPL.df.filter(pl.col("code")==code)

code,date,unsold_margin,purchase_margin,margin_ratio
i64,date,i64,i64,f64
